In [1]:
# On terminal: conda activate python38

In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
import ipynb.fs.defs.functions as fct
import pickle

import warnings
warnings.filterwarnings("error")

In [8]:
# Load IDs
with open('uniqueIDs.pkl', 'rb') as f:
    uniqueIDs = pickle.load(f)

# Fit mod5 for each participant

In [9]:
# Model settings

# Functions
value_fct = fct.rescorla_wagner_shrinking_alpha
dec_fct = fct.my_softmax_scipy

# Store everything
mod_info = {}
mod_info['name'] = 'model5'
mod_info['value_fct'] = value_fct.__name__
mod_info['dec_fct'] = dec_fct.__name__
mod_info['param_names'] = ['v0', 'alpha', 'alpha_t', 'beta']
#print(mod['value_fct'].__name__)

# save
all_users_folder = 'data/all_users/mod5/'
file_name = all_users_folder+'mod_parameters.pkl'
with open(file_name, 'wb') as f:
    pickle.dump(mod_info, f)

In [10]:
run_ = True

# Folder
all_users_folder = 'data/all_users/mod5/'
    
if run_:
    
    # Parameter range for initial guess 
    # order of mod['param_names']:
    # ['v0', 'alpha', 'alpha_t', 'beta']
    param_lower_bound = [-10, 0, 0, 0]
    param_upper_bound = [ 10, 1, 1, 3]

    # Fit
    all_users = {}
    p_hit_per_trial = pd.DataFrame([])
    ev_per_trial = pd.DataFrame([])
    
    for n_part,ID in enumerate(uniqueIDs): 

        # Get data
        user_folder = 'data/user_' + ID + '/'
        df2_cf = pd.read_pickle(user_folder + 'df2_cf.pkl')
        isHit_all_cues, fbs_all_cues, trialNo_all_cues = fct.extract_hits_fbs(df2_cf)

        # Create a new Model object
        mod = fct.Model(mod_name = mod_info['name'],
                     value_fct = value_fct, 
                     dec_fct = dec_fct, 
                     param_names = mod_info['param_names'])

        # Input data to model
        mod.set_data(ID, fbs_all_cues, isHit_all_cues, trialNo_all_cues)

        # Fit model
        mod.fit(param_lower_bound, param_upper_bound, n_iterations=5)

        # Nested dictionnary user data
        all_users[n_part] = {}
        all_users[n_part]['ID']=mod.ID
        all_users[n_part]['nLL']=mod.nLL
        all_users[n_part]['Ntrials']=mod.Ntrials
        all_users[n_part]['Nparams']=len(mod.param_names)
        for i in range(0,len(mod.param_names)):
            all_users[n_part][mod.param_names[i]]=mod.param_values[i]
        
        # Concatenated model predictions: p hit
        tmp = pd.DataFrame(mod.p_hit).transpose()
        tmp.columns = tmp.columns+1
        tmp = tmp.reset_index().rename(columns={'index': 'Cue'})
        tmp.insert(0,'ID',ID)
        p_hit_per_trial = pd.concat([tmp, p_hit_per_trial], axis=0)
        
        # Concatenated model predictions: EVs
        tmp = pd.DataFrame(mod.v).transpose()
        tmp.columns = tmp.columns+1
        tmp = tmp.reset_index().rename(columns={'index': 'Cue'})
        tmp.insert(0,'ID',ID)
        ev_per_trial = pd.concat([tmp, ev_per_trial], axis=0)
    
    # Save mod LLs and parameter values
    mod_fit = pd.DataFrame(all_users).transpose()
    mod_fit.to_pickle(all_users_folder+'mod_param_fits.pkl')
    
    # Save mod predictions
    p_hit_per_trial = p_hit_per_trial.sort_values(by='ID').reset_index(drop=True)
    p_hit_per_trial.to_pickle(all_users_folder+'mod_p_hit_per_trial.pkl')
    ev_per_trial = ev_per_trial.sort_values(by='ID').reset_index(drop=True)
    ev_per_trial.to_pickle(all_users_folder+'mod_ev_per_trial.pkl')
    

In [11]:
mod_fit

,ID,nLL,Ntrials,Nparams,v0,alpha,alpha_t,beta
0,001,51.917746,112,4,-0.508049,0.0,0.029011,3.0
1,003,53.289788,112,4,0.141746,0.026828,0.0,3.0
2,006,54.445219,112,4,0.771043,0.157212,0.101785,0.930084
3,007,48.908147,112,4,0.714076,0.0,0.076235,1.958113
4,008,44.242888,112,4,0.343681,0.033001,0.0,3.0
...,...,...,...,...,...,...,...,...
165,453,48.537262,112,4,10.0,0.047523,0.350352,0.984306
166,462,45.068864,112,4,0.080988,0.036538,0.0,3.0
167,469,8.934709,112,4,0.191054,0.0,0.18747,3.0
168,470,49.787823,112,4,0.169967,0.0,0.0415,3.0
